In [1]:
from plantillasArquitectura import Datos as d
from scipy.stats import norm

import numpy as np

In [2]:
lenses = d.Datos('./plantillasArquitectura/datos/conjunto_datos_lentillas.txt')


In [3]:
lenses.datos
datosTrain=lenses.datos
atributosDiscretos = lenses.nominalAtributos
print(lenses.nombreAtributos)
print(lenses.tipoAtributos)
print(lenses.nominalAtributos)
print(lenses.diccionarios)
print(lenses.datos)

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
['Nominal', 'Nominal', 'Nominal', 'Nominal', 'Nominal']
[True, True, True, True, True]
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]


In [10]:
print(lenses.diccionarios[-1])
for key in lenses.diccionarios[-1]:
    print(key)
print(lenses.diccionarios[-1].keys())

{'1': 0, '2': 1, '3': 2}
1
2
3


NameError: name 'keys' is not defined

In [175]:
print(datosTrain.shape)
print(type(datosTrain))
prioris = dict()
nDatos = len(datosTrain)
classIdx = np.size(datosTrain,1)-1 
for dato in datosTrain:
    clase = dato[classIdx]
    if clase not in prioris.keys():
        prioris[clase] = 1/nDatos
    else :
        prioris[clase] += 1/nDatos
classes = list((datosTrain[:, -1]))
third = list(set(datosTrain[:, 2]))

filteredColumn = datosTrain[datosTrain[:,-1]==classes[0]][:,3]
print(filteredColumn)
mean = np.mean(filteredColumn)
std = np.std(filteredColumn)
gen = norm(mean, std)
print(gen.pdf(0.5))                
np.sum((datosTrain[:,2]==third[0] ) & (datosTrain[:,-1]==classes[0]))

(24, 5)
<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.]
0.752843580387011


7

In [ ]:
    # Initialize the posteriori numerator as the priori probability for clase
    verodotpriori = 1
    # Initialize the posteriori numerator as the priori probability for clase
    verodotpriori = 1
    # Initialize the posteriori numerator as the priori probability for clase
    verodotpriori = 1
    # Initialize the posteriori numerator as the priori probability for clase
    verodotpriori = 1


In [139]:
from scipy.stats import norm

# There are len - 1 attributes, since last element in atributosDiscretos, the class, does not correspond to an attribute
nAtributos = len(atributosDiscretos)-1
# We extract a set with all classes in the data
classes = set(datosTrain[:, -1])

# We create a list where each element of the list corresponds to the i-th attribute,
# and for each attribute a dictionary with classes as keys is created
# If the attribute is continuous, then for each class key the appropriate normal distribution is assigned as a value
# Otherwise, for each class key a new dictionary is created, which has the attribute values as keys,
#            and for each value key,the number of data elements with the current class and current value is assigned as value

attrTables = []
for i in range(nAtributos-1):
        # For each attribute, a dictionary with all classes as values
        classesTable = dict()
            
        for clase in classes:
            # We extract a set with all attribute values for the i-th attribute
            attrValues = set(datosTrain[:,i])
            # If i-th attribute discrete:
            # For each class, a dictionary with the i-th attribute values as keys
            classesTable[clase] = dict()            
            if(atributosDiscretos[i]):
                for value in attrValues:
                    # We count the number of elements of class clase out of the elements 
                    # where the i-th attribute has value value
                    count = np.sum((datosTrain[:,i]==value ) & (datosTrain[:,-1]==clase))
                    #if count == 0: 
                    #    AQUI SE HARIA LAPLACE: activar una flag que, una vez recorridos todos los values, sume uno a todos los dics
                    #    Ineficiente a tope pero al menos no es larguisimo, nValuesxNclasses no deberia ser un valor muy grande    
                    classesTable[clase][value] = count

            # If i-th attribute continuous:
            else:
                # We create an array with the i-th attribute values of data where class == clase
                filteredColumn = datosTrain[datosTrain[:,-1]==clase][:, i]
                # We extract mean and variance of the i-th column
                mean = np.mean(filteredColumn)
                std = np.std(filteredColumn)
                classesTable[clase] = norm(mean, std)
                
        attrTables.append(classesTable)   
        

In [160]:
NBTables = attrTables
prioris
sum(attrTables[0][2].values())

15

In [168]:
testData = lenses.datos

pred = []
for data in testData:
    maxClass = [clase, 0]
    for clase in classes:    
        # Initialize the posteriori numerator as the priori probability for clase
        verodotpriori = prioris[clase]
        # Now we multiply by each P(attrN == valueofattrNindata | clase)
        nAtributos = len(atributosDiscretos)-1
        for i in range(nAtributos-1):
            # Value of the i-th attribute in the given testData element 
            value = data[i]
            # We search in NBTables: 
            # take the i-th position of the array, corresponding to the dictionary of the i-th attribute
            if atributosDiscretos[i]:
                # inside, if the attribute is discrete, the dictionary corresponding to the 'clase' key 
                # and from there, the key 'value' (which is number of occurrences of class = clase and ithattribute = value)
                nOccurrences = NBTables[i][clase][value]
                # And divide by the number of occurrences of the other values given class = clase
                vero = nOccurrences/sum(attrTables[i][clase].values())
            else :
                # if the attribute is continuous, take the distribution stored in the 'clase' key
                # and calculate the pdf of the ith attribute being the value that our testData element has
                vero = NBTables[i][clase].pdf(value)
            verodotpriori *= vero
        
        # If the last calculated numerator is greater than the previous max, update the class and its numerator
        if verodotpriori > maxClass[1]:
            maxClass = [clase, verodotpriori]
    # We append to the pred array the class predicted for the testData element we are testing
    pred.append(maxClass[0])

In [167]:
pred

[2.0,
 2.0,
 0.0,
 0.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0]